In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.webdriver import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Setup Chrome WebDriver
options = Options()
options.add_argument("--headless")  # Run in headless mode (optional)
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to extract table data from the current page
def extract_table_data():
    table_rows = driver.find_elements(By.CSS_SELECTOR, 'tr.sc-635c3a22-0.bFRTed')
    data = []

    for row in table_rows:
        try:
            # Extract the necessary columns from the row
            name = row.find_element(By.CSS_SELECTOR, 'td.sc-8a4e62dd-0.bRkVpI').text.strip()
            tier = row.find_element(By.CSS_SELECTOR, 'td.sc-8a4e62dd-0.fSPZjY div').text.strip()
            latest_deal = row.find_element(By.CSS_SELECTOR, 'td.sc-8a4e62dd-0.dvhjZw').text.strip()
            portfolio = row.find_element(By.CSS_SELECTOR, 'td.sc-8a4e62dd-0.fSPZjY p').text.strip()
            retail_roi = row.find_element(By.CSS_SELECTOR, 'td.sc-8a4e62dd-0.cNMbGg p').text.strip()
            focus_area = row.find_element(By.CSS_SELECTOR, 'td.sc-8a4e62dd-0.dvhjZw div').text.strip()
            
            # Append the row data to the list
            data.append([name, tier, latest_deal, portfolio, retail_roi, focus_area])
        except Exception as e:
            print(f"Error extracting data from row: {e}")

    return data

# Function to simulate clicking the "Next page" button
def click_next_page():
    try:
        next_button = driver.find_element(By.CLASS_NAME, "sc-b4e30450-0.sc-8a92c9ab-1.jPDtNl.cwhYhX")
        next_button.click()
        print("Next page clicked.")
    except Exception as e:
        print(f"Error clicking next page button: {e}")

# Open the initial page
driver.get("https://cryptorank.io/funds?page=1&rows=100")

# Wait for the page to load
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "sc-8b138daa-1.gEBUGZ")))

# Extract data from the first page
print("Scraping data from page 1...")
data_page_1 = extract_table_data()

# Click the "Next page" button and wait for the page to load
click_next_page()

# Wait for the new page to load
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "sc-8b138daa-1.gEBUGZ")))

# Extract data from the second page
print("Scraping data from page 2...")
data_page_2 = extract_table_data()

# Combine data from both pages
all_data = data_page_1 + data_page_2

# Create a DataFrame from the combined data
columns = ['Name', 'Tier', 'Latest Deal', 'Portfolio', 'Retail ROI', 'Focus Area']
df = pd.DataFrame(all_data, columns=columns)

# Show the first 10 rows of the DataFrame as a sample
print(df.head(10))

# Save the DataFrame to a CSV file (optional)
df.to_csv("scraped_data.csv", index=False)

# Close the driver after completion
driver.quit()

print("Data scraping completed and saved to 'scraped_data.csv'.")


Scraping data from page 1...
Error extracting data from row: Message: no such element: Unable to locate element: {"method":"css selector","selector":"td.sc-8a4e62dd-0.cNMbGg p"}
  (Session info: chrome=134.0.6998.179); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0115C7F3+24435]
	(No symbol) [0x010E2074]
	(No symbol) [0x00FB06E3]
	(No symbol) [0x00FF8B39]
	(No symbol) [0x00FF8E8B]
	(No symbol) [0x00FEE1F1]
	(No symbol) [0x0101D804]
	(No symbol) [0x00FEE114]
	(No symbol) [0x0101DA34]
	(No symbol) [0x0103F20A]
	(No symbol) [0x0101D5B6]
	(No symbol) [0x00FEC54F]
	(No symbol) [0x00FED894]
	GetHandleVerifier [0x014670A3+3213347]
	GetHandleVerifier [0x0147B0C9+3295305]
	GetHandleVerifier [0x0147558C+3271948]
	GetHandleVerifier [0x011F7360+658144]
	(No symbol) [0x010EB27D]
	(No symbol) [0x010E8208]
	(No symbol) [0x010E83A9]
	(No symbol) [0x010DAAC0]
	BaseThread

In [3]:
df

,Fund Name,Tier,Latest Deal,Portfolio,Retail ROI,Focus Area
0,Delphi Ventures,1,Ambient\n$7.20M\n31 Mar 2025,160,161.06x,#\nArtificial Intell...\n+6
1,a16z CSX,1,Mahojin\nN/A\n31 Mar 2025,34,,#\nArtificial Intell...\n+6
2,Polychain Capital,1,Warlock\n$8.00M\n27 Mar 2025,206,15.44x,#\nStablecoin Protoc...\n+6
3,Sandeep Nailwal,1,Capx AI\n$3.14M\n26 Mar 2025,129,0.37x,#\nArtificial Intell...\n+6
4,HashKey Capital,1,Spot Zero\n$4.50M\n25 Mar 2025,260,1.71x,#\nPerpetuals\n+6
...,...,...,...,...,...,...
10895,Avalanche Foundat...,2,KnidosLabs\nN/A\n21 Feb 2025,21,0.09x,#\nNFT elements\n+6
10896,CMS Holdings,2,Hedgemony\nN/A\n18 Feb 2025,189,7.16x,#\nDEX\n+6
10897,GV Google Venture...,2,Blockaid\n$50.00M\n18 Feb 2025,19,0.48x,#\nPayments\n+6
10898,eGirl Capital,2,Cygnus Finance\n$20.00M\n18 Feb 2025,14,,#\nGovernance\n+6


In [4]:
duplicate_funds = df[df.duplicated(subset='Fund Name', keep=False)]
print(duplicate_funds)

                  Fund Name Tier                           Latest Deal  \
0           Delphi Ventures    1          Ambient\n$7.20M\n31 Mar 2025   
1                  a16z CSX    1             Mahojin\nN/A\n31 Mar 2025   
2         Polychain Capital    1          Warlock\n$8.00M\n27 Mar 2025   
3           Sandeep Nailwal    1          Capx AI\n$3.14M\n26 Mar 2025   
4           HashKey Capital    1        Spot Zero\n$4.50M\n25 Mar 2025   
...                     ...  ...                                   ...   
10895  Avalanche Foundat...    2          KnidosLabs\nN/A\n21 Feb 2025   
10896          CMS Holdings    2           Hedgemony\nN/A\n18 Feb 2025   
10897  GV Google Venture...    2        Blockaid\n$50.00M\n18 Feb 2025   
10898         eGirl Capital    2  Cygnus Finance\n$20.00M\n18 Feb 2025   
10899       Solana Ventures    2         BitRobot\n$6.00M\n14 Feb 2025   

      Portfolio Retail ROI                   Focus Area  
0           160    161.06x  #\nArtificial Intell...\n

In [11]:
import requests
import pandas as pd
from time import sleep
from tqdm.notebook import tqdm  # for progress bars

# %% [markdown]
# ## Configuration

# %%
BASE_URL = "https://api.cryptorank.io/v0/funds/table/"
OUTPUT_FILE = "funds_details.csv"
TOTAL_RECORDS = 10879
BATCH_SIZE = 100
DELAY_BETWEEN_REQUESTS = 1  # seconds

# %% [markdown]
# ## Data Fetching Function

# %%
def fetch_funds_data():
    all_data = []
    
    # Create progress bar
    pbar = tqdm(total=TOTAL_RECORDS, desc="Fetching data")
    
    for offset in range(0, TOTAL_RECORDS, BATCH_SIZE):
        try:
            # Make API request
            response = requests.get(
                BASE_URL,
                params={
                    'limit': BATCH_SIZE,
                    'offset': offset
                }
            )
            response.raise_for_status()
            
            # Parse JSON data
            data = response.json()
            
            if data.get('data'):
                all_data.extend(data['data'])
                pbar.update(len(data['data']))
            else:
                print(f"\nNo data received at offset {offset}")
                
            # Respect rate limits
            sleep(DELAY_BETWEEN_REQUESTS)
            
        except Exception as e:
            print(f"\nError at offset {offset}: {str(e)}")
            break
    
    pbar.close()
    return all_data

# %% [markdown]
# ## Execute Data Fetch

# %%
print("Starting data fetch...")
funds_data = fetch_funds_data()
print(f"\nFetched {len(funds_data)} records")

# %% [markdown]
# ## Convert to DataFrame and Save

# %%
if funds_data:
    # Create DataFrame
    df = pd.DataFrame(funds_data)
    
    # Display preview
    print("\nData preview:")
    display(df.head())
    
    # Save to CSV
    df.to_csv(OUTPUT_FILE, index=False)
    print(f"\nData saved to {OUTPUT_FILE}")
    
    # Show summary
    print("\nSummary:")
    print(f"Total records: {len(df)}")
    print(f"Columns: {list(df.columns)}")
else:
    print("No data was fetched")

Starting data fetch...


Fetching data:   0%|          | 0/10879 [00:00<?, ?it/s]


Fetched 10879 records

Data preview:


,slug,name,logo,tier,type,location,latestDeal,portfolio,portfolioData,retailRoi,...,focusAreaData,avgRoundRaise,avgRoundRaiseData,preferredStage,preferredStageData,fundingRounds,leadInvestments,mainFundingCountry,mainFundingCountryData,twitterData
0,hash-key-capital,HashKey Capital,https://img.cryptorank.io/funds/60x60.hashkey ...,1.0,Venture,Hong Kong,"{'key': 'meet-48', 'name': 'MEET48', 'icon': '...",261.0,"[{'key': 'secret', 'name': 'Secret Network', '...",1.803606,...,"[{'tag': 'Perpetuals', 'count': 3, 'percent': ...","{'raiseFrom': 3000000, 'raiseTo': 10000000, 'p...","[{'raiseFrom': 0, 'raiseTo': 1000000, 'percent...",SEED,"[{'percent': 45, 'type': 'SEED'}, {'percent': ...",224.0,21.0,United States,"[{'country': 'United States', 'count': 12}, {'...",None
1,coinbase-ventures,Coinbase Ventures,https://img.cryptorank.io/funds/60x60.coinbase...,1.0,Venture,United States,"{'key': 'momentum-finance', 'name': 'Momentum'...",423.0,"[{'key': 'ftx-token', 'name': 'FTX Token', 'ic...",3.248852,...,"[{'tag': 'Payments', 'count': 4, 'percent': 22...","{'raiseFrom': 3000000, 'raiseTo': 10000000, 'p...","[{'raiseFrom': 0, 'raiseTo': 1000000, 'percent...",SEED,"[{'percent': 35, 'type': 'SEED'}, {'percent': ...",402.0,25.0,United States,"[{'country': 'United States', 'count': 14}, {'...","{'twitterUsername': 'cbventures', 'followersCo..."
2,delphi-ventures,Delphi Ventures,https://img.cryptorank.io/funds/60x60.delphi_v...,1.0,Venture,None,"{'key': 'ambient', 'name': 'Ambient', 'icon': ...",160.0,"[{'key': 'sentient', 'name': 'Sentient', 'icon...",163.101180,...,"[{'tag': 'Artificial Intelligence (AI)', 'coun...","{'raiseFrom': 3000000, 'raiseTo': 10000000, 'p...","[{'raiseFrom': 0, 'raiseTo': 1000000, 'percent...",SEED,"[{'percent': 35, 'type': 'SEED'}, {'percent': ...",140.0,27.0,United States,"[{'country': 'United States', 'count': 11}, {'...","{'twitterUsername': 'Delphi_Ventures', 'follow..."
3,circle,Circle,https://img.cryptorank.io/funds/60x60.circle16...,1.0,Venture,United States,"{'key': 'momentum-finance', 'name': 'Momentum'...",99.0,"[{'key': 'sui', 'name': 'Sui', 'icon': 'https:...",4.259379,...,"[{'tag': 'Lending', 'count': 3, 'percent': 18....","{'raiseFrom': 3000000, 'raiseTo': 10000000, 'p...","[{'raiseFrom': 1000000, 'raiseTo': 3000000, 'p...",SEED,"[{'percent': 60, 'type': 'SEED'}, {'percent': ...",95.0,3.0,United States,"[{'country': 'United States', 'count': 15}, {'...",None
4,a16z-csx,a16z CSX,https://img.cryptorank.io/funds/60x60.a_16_z_c...,1.0,Incubator,None,"{'key': 'ambient', 'name': 'Ambient', 'icon': ...",34.0,"[{'key': 'opacity-network', 'name': 'Opacity N...",NaN,...,"[{'tag': 'Artificial Intelligence (AI)', 'coun...","{'raiseFrom': 3000000, 'raiseTo': 10000000, 'p...","[{'raiseFrom': 0, 'raiseTo': 1000000, 'percent...",Incubation,"[{'percent': 40, 'type': 'Incubation'}, {'perc...",38.0,4.0,United States,"[{'country': 'United States', 'count': 4}, {'c...",None



Data saved to funds_details.csv

Summary:
Total records: 10879
Columns: ['slug', 'name', 'logo', 'tier', 'type', 'location', 'latestDeal', 'portfolio', 'portfolioData', 'retailRoi', 'focusArea', 'focusAreaData', 'avgRoundRaise', 'avgRoundRaiseData', 'preferredStage', 'preferredStageData', 'fundingRounds', 'leadInvestments', 'mainFundingCountry', 'mainFundingCountryData', 'twitterData']
